# Refactoring our simulator classes to something more modular

In [ ]:
import os
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import hcipy
from astropy.io import fits

from pastis.config import CONFIG_PASTIS
import pastis.util
from pastis.e2e_simulators.luvoir_imaging import SegmentedTelescope, LuvoirAPLC, LuvoirA_APLC, SegmentedAPLC

In [ ]:
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')
optics_input = os.path.join(pastis.util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
design = 'small'
luvoir = LuvoirAPLC(optics_input, design, sampling)

In [ ]:
luvoir.seg_pos.size

In [ ]:
ob_luvoir = luvoir.calc_out_of_band_wfs()
plt.imshow(ob_luvoir.intensity.shaped)

## Let's look at SegmentedTelescope first.

In [ ]:
input_dir = optics_input
apod_design = design

apod_dict = {'small': {'pxsize': 1000, 'fpm_rad': 3.5, 'fpm_px': 150, 'iwa': 3.4, 'owa': 12.,
                            'fname': '0_LUVOIR_N1000_FPM350M0150_IWA0340_OWA01200_C10_BW10_Nlam5_LS_IDD0120_OD0982_no_ls_struts.fits'},
                  'medium': {'pxsize': 1000, 'fpm_rad': 6.82, 'fpm_px': 250, 'iwa': 6.72, 'owa': 23.72,
                             'fname': '0_LUVOIR_N1000_FPM682M0250_IWA0672_OWA02372_C10_BW10_Nlam5_LS_IDD0120_OD0982_no_ls_struts.fits'},
                  'large': {'pxsize': 1000, 'fpm_rad': 13.38, 'fpm_px': 400, 'iwa': 13.28, 'owa': 46.88,
                            'fname': '0_LUVOIR_N1000_FPM1338M0400_IWA1328_OWA04688_C10_BW10_Nlam5_LS_IDD0120_OD0982_no_ls_struts.fits'}}
imlamD = 1.2 * apod_dict[apod_design]['owa']

wvln = CONFIG_PASTIS.getfloat('LUVOIR', 'lambda') * 1e-9    # m
diameter = CONFIG_PASTIS.getfloat('LUVOIR', 'diameter')     # m
lam_over_d = wvln / diameter

pupil_grid = hcipy.make_pupil_grid(dims=apod_dict[apod_design]['pxsize'], diameter=diameter)

# Load segmented aperture
aper_path = CONFIG_PASTIS.get('LUVOIR', 'aperture_path_in_optics')
pup_read = hcipy.read_fits(os.path.join(input_dir, aper_path))
aperture = hcipy.Field(pup_read.ravel(), pupil_grid)

# Load apodizer
apod_path = os.path.join('luvoir_stdt_baseline_bw10', apod_design + '_fpm', 'solutions',
                         apod_dict[apod_design]['fname'])
apod_read = hcipy.read_fits(os.path.join(input_dir, apod_path))
apodizer = hcipy.Field(apod_read.ravel(), pupil_grid)

# Load Lyot Stop
ls_fname = CONFIG_PASTIS.get('LUVOIR', 'lyot_stop_path_in_optics')
ls_read = hcipy.read_fits(os.path.join(input_dir, ls_fname))
lyot_stop = hcipy.Field(ls_read.ravel(), pupil_grid)

# Load indexed segmented aperture
aper_ind_path = CONFIG_PASTIS.get('LUVOIR', 'indexed_aperture_path_in_optics')
aper_ind_read = hcipy.read_fits(os.path.join(input_dir, aper_ind_path))
aper_ind = hcipy.Field(aper_ind_read.ravel(), pupil_grid)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(input_dir, aper_ind_path))
nseg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i + 1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
poslist = np.transpose(np.array(poslist))
seg_pos = hcipy.CartesianGrid(hcipy.UnstructuredCoords(poslist))
seg_pos = seg_pos.scaled(diameter)

seg_diameter_circumscribed = 2 / np.sqrt(3) * 1.2225    # m

# Create a focal plane mask
samp_foc = apod_dict[apod_design]['fpm_px'] / (apod_dict[apod_design]['fpm_rad'] * 2)
focal_grid_fpm = hcipy.make_focal_grid_from_pupil_grid(pupil_grid=pupil_grid, q=samp_foc, num_airy=apod_dict[apod_design]['fpm_rad'], wavelength=wvln)
fpm = 1 - hcipy.circular_aperture(2*apod_dict[apod_design]['fpm_rad'] * lam_over_d)(focal_grid_fpm)

# Create a focal plane grid for the detector
focal_det = hcipy.make_focal_grid_from_pupil_grid(pupil_grid=pupil_grid, q=sampling, num_airy=imlamD, wavelength=wvln)

In [ ]:
seg = SegmentedTelescope(wvln, diameter, aperture, aper_ind, seg_pos, seg_diameter_circumscribed, focal_det, sampling, imlamD)

In [ ]:
# Create high-order (ripple) mode mirror
n_ripples = 5    # need to use odd number
seg.create_ripple_mirror(n_ripples)

In [ ]:
# Create a good ol' continuous DM
n_acts_across = 15 
seg.create_continuous_deformable_mirror(n_acts_across)

In [ ]:
# Create low-order (Zernike) mode mirror
n_modes_zernikes = 8
seg.create_global_zernike_mirror(n_modes_zernikes)

In [ ]:
# Create segmented Harris mode mirror
# !! THIS CELL TAKES QUITE A WHILE TO RUN !!
fpath = '/Users/ilaginja/repos/PASTIS/Sensitivities2.xlsx'    # path to Harris spreadsheet
pad_orientations = np.pi / 2 * np.ones(120)
seg.create_segmented_harris_mirror(fpath, pad_orientations)

In [ ]:
# Create multi mode segmented mirror
# !! THIS CELL TAKES QUITE A WHILE TO RUN !!
n_modes_segs = 5
seg.create_segmented_mirror(n_modes_segs)

**CREATE MIRRORS ONLY ONCE**

In [ ]:
# For usage with ripple mirror
new_command = np.zeros(n_ripples*n_ripples)
#new_command[12] = 2e-8
seg.ripple_mirror.actuators = new_command

In [ ]:
# For usage with continuous deformable mirror
new_command = np.zeros(n_acts_across*n_acts_across)
new_command[66] = 2e-8
new_command[77] = 2e-8
#new_command[147] = 2e-7
#new_command[84] = 2e-7
#new_command[45] = -2e-7
#new_command[34] = 2e-7
#new_command[24] = 2e-7
#new_command[217] = -2e-7
#new_command[187] = -2e-7
#new_command[123] = 2e-7
#new_command[105] = -2e-7
#new_command[173] = 2e-7
seg.dm.actuators = new_command

In [ ]:
# For usage with Zernike mode mirror
new_command = np.zeros(n_modes_zernikes)
#new_command[7] = 2e-8
seg.zernike_mirror.actuators = new_command

In [ ]:
# For usage with segmented Harris mode mirror
new_command = np.zeros(luvoir.harris_sm.num_actuators)
print(new_command.shape)
#new_command[18] = 1e-8
#new_command[37] = 2e-8
seg.harris_sm.actuators = new_command

In [ ]:
# For usage with multi mode segmented mirror
new_command = np.zeros(120*n_modes_segs)
#new_command[4] = 2e-8
#new_command[51] = 2e-4
#new_command[346] = 2e-8
seg.sm.actuators = new_command

In [ ]:
image, inter = seg.calc_psf(display_intermediate=True, return_intermediate='efield')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image.intensity.shaped, norm=LogNorm(), origin='lower')

In [ ]:
inter

In [ ]:
plt.imshow(inter['active_pupil'].phase.shaped, origin='lower')

In [ ]:
obwfs = seg.calc_out_of_band_wfs()

In [ ]:
plt.imshow(obwfs.intensity.shaped)
plt.colorbar()

## Now on to the SegmentedAPLC

In [ ]:
aplc = SegmentedAPLC(apodizer, lyot_stop, fpm, apod_dict[apod_design]['fpm_rad'], apod_dict[apod_design]['iwa'],
                     apod_dict[apod_design]['owa'], wvln=wvln, diameter=diameter, aper=aperture,
                     indexed_aper=aper_ind, seg_pos=seg_pos, seg_diameter=seg_diameter_circumscribed,
                     focal_grid=focal_det, sampling=sampling, imlamD=imlamD)

In [ ]:
aplc_im = aplc.calc_psf(display_intermediate=True)

In [ ]:
lowfs_im = aplc.calc_low_order_wfs()
plt.imshow(lowfs_im.intensity.shaped)

## And finally, the new LUVOIR A with an APLC simulator class

In [ ]:
luv = LuvoirA_APLC(optics_input, design, sampling)

In [ ]:
# Create low-order (Zernike) mode mirror
n_modes_zernikes = 15
luv.create_global_zernike_mirror(n_modes_zernikes)

In [ ]:
# For usage with Zernike mode mirror
new_command = np.zeros(n_modes_zernikes)
new_command[12] = 2e-8
new_command[4] = 2e-8
luv.zernike_mirror.actuators = new_command

In [ ]:
luv_img, luv_direct = luv.calc_psf(display_intermediate=True, ref=True)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(luv_img.shaped/luv_direct.max(), norm=LogNorm(), cmap='inferno')
plt.colorbar()